In [1]:
import pandas as pd
import os
import re
import glob
from Bio import Entrez, Medline
import sys, os
import numpy as np
from bs4 import BeautifulSoup

In [2]:
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

In [3]:
# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [4]:
def get_nxml_paths():

    liste_zipped_files = glob.glob("../articles/xml_files/*.nxml")
    #print(liste_zipped_files)
    return liste_zipped_files

In [5]:
def change_type_df_column(df):
    import numpy as np
    
    # decode columns as utf-8
    
    col_list = ["PMC"]
    for i in col_list:
        bytelist = df[i].tolist()
        stringlist = [x[0].decode('utf-8') for x in bytelist]
        df[i] = stringlist

    col_list_2 = ["FAU","AU","AID","AUID"]

    for i in col_list_2:
        liste = []
        bytelist = df[i].tolist()

        for j in bytelist:
            if type(j) != float:
                stringlist = [x.decode('utf-8') for x in j]
                liste.append(stringlist)
            else:
                liste.append([])
        df[i] = liste
    return df

In [6]:
def retrieve_pmc_list():
    liste_zipped_files = glob.glob("../articles/xml_files/*.nxml")
    flat_list = []
    
    # retieve metadata for every downloaded full text
    
    for i in liste_zipped_files:
        i = i.replace(".nxml","")
        flat_list.append(i[-10:])
    
    
    handle = Entrez.efetch(db="pmc", id=flat_list, rettype="medline", retmode="json")
    records = Medline.parse(handle)
    data_list = []
    for record in records:
        data_list.append(record)
        
    # create a dataframe from the retrieved metadata
    
    df = pd.DataFrame(data_list)
    df.columns = df.columns.astype(str)


    # drop unwanted columns and decode as utf-8
    
    df_filtered = df[["PMC","FAU","AU","AID","AUID"]]

    df_filtered = change_type_df_column(df_filtered)


    return df_filtered

In [7]:
def extract_body_content(df_metadata):
    liste_zipped_files =  get_nxml_paths()
    liste_text = []
    liste_name = []
    
    # extract the content out of the xml-files
    
    for i in liste_zipped_files:
        temp_text_list = []
        with open(i, "r", encoding="utf-8") as f:
            article = f.read()
            f.close()
        soup = BeautifulSoup(article, "lxml")
        for text in soup.find_all("p"):
            temp_text_list.append(text)

        name_str = "PMC"+soup.find("article-id", {"pub-id-type": "pmc"}).text
        liste_name.append(name_str)
        liste_text.append(temp_text_list)
    
    #merge the dataframe with the metadata with the dataframe containing the full texts
    
    df = pd.DataFrame()
    df["PMCID"] = liste_name
    df["text"] = liste_text
   
    df = df.merge(df_metadata, left_on="PMCID", right_on="PMC")
    
    # delete tags within the fulltexts
    
    df["text"] = df["text"].apply(lambda x: re.sub(r'<.*?>', '', str(x)))
    return df

In [8]:
Entrez.api_key = ""
Entrez.email = ""
df_metadata = retrieve_pmc_list()

/tmp/ipykernel_2553/3387890410.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = stringlist
/tmp/ipykernel_2553/3387890410.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = liste


In [9]:
df = extract_body_content(df_metadata)

In [10]:
df = df.drop("PMC", axis=1)

In [11]:
df.to_csv("/path/to/store/dataframe")